In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/My Drive/Classroom/Big Data Analytics - Spring 2021/

/content/gdrive/My Drive/Classroom/Big Data Analytics - Spring 2021


In [3]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import re

In [4]:
df=pd.read_csv('selected_data.csv')

In [ ]:
#zakladne:basic,stredoskolske:high school,vysokoskolske:university,ucnovske:apprentice

In [6]:
def categorizedEducationLevels(column):
    categorized_edu_list=[]
    for val in column:
      temp=val.split()
      #print("THis os temp ",temp)
      #print("this is res ",set(temp).intersection(["zakladne","skole","skolka"]))
      if  set(temp).intersection(["zakladne","skole","skolka"]) !=set():
        categorized_edu_list.append("zakladne")

      elif  set(temp).intersection(["stredoskolske","strednej","stredna","stredne"]) !=set(): 
        categorized_edu_list.append("stredoskolske")

      elif  set(temp).intersection(["vysokoskolske","bakalarske"]) !=set():
        categorized_edu_list.append("vysokoskolske")

      elif  set(temp).intersection(["ucnovske"]) !=set(): 
        categorized_edu_list.append("ucnovske")

      else:
        categorized_edu_list.append("")

    return categorized_edu_list




In [7]:
categorized_edu_list=categorizedEducationLevels(df.level_of_education)

In [8]:
df.level_of_education=categorized_edu_list

In [9]:
df.drop(df[df['level_of_education'] =='' ].index, inplace = True)

In [10]:
input_features=df[['age','level_of_education']]
list_input_features=input_features.values.tolist()

In [11]:
encoder = TransactionEncoder()
encoder_apriori = encoder.fit(list_input_features).transform(list_input_features)
df_apriori = pd.DataFrame(encoder_apriori, columns=encoder.columns_)


In [12]:
from mlxtend.frequent_patterns import apriori

freq_itemset=apriori(df_apriori, min_support=0.01, use_colnames=True)

In [13]:
from mlxtend.frequent_patterns import association_rules

rules=association_rules(freq_itemset, metric="lift", min_threshold=0)

#Visualization 1

In [14]:
vis1=rules

In [15]:
vis1=vis1[ vis1.lift<1]
vis1.lift=vis1.lift.round(decimals=6)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [16]:
vis1=vis1.drop_duplicates(subset =["lift"] )

In [17]:
#def convertToProperFormat(dframe):
age_col=[]
edu_col=[]

for row in vis1.itertuples():
    a,=row.antecedents
    c,=row.consequents
    if  any(d.isdigit() for d in a[0]):
      age_col.append(a)
      edu_col.append(c)
    else:
      age_col.append(c)
      edu_col.append(a)    




 # return new_ant,new_con

In [18]:
plot_frame=pd.DataFrame(list(zip(age_col,edu_col,vis1.lift)),columns=["age","education_level","lift"])

In [20]:
import plotly.express as px

fig = px.bar(plot_frame, x="education_level", y="lift", color='age', 
             labels=dict(age="Age Group",education_level="Level of Education Completed"),
            color_discrete_sequence=px.colors.qualitative.Dark2)
fig.update_layout(title_text="Correlation Analysis Between Education and Age ")
fig.show()

#Visualization 2

In [34]:
vis2=rules
vis2=vis2[ vis2.lift>1]
vis2.lift=vis2.lift.round(decimals=6)
vis2=vis2.drop_duplicates(subset =["lift"] )


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
vis2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(zakladne),(0-20),0.388976,0.521945,0.335863,0.863454,1.654299,0.132839,3.501048
8,(stredoskolske),(21-40),0.455785,0.436467,0.289704,0.635614,1.456272,0.090769,1.546529
10,(21-40),(ucnovske),0.436467,0.045919,0.026652,0.061064,1.329828,0.006610,1.016130
16,(stredoskolske),(41-60),0.455785,0.040240,0.027098,0.059454,1.477494,0.008758,1.020429


In [35]:
age_col=[]
edu_col=[]

for row in vis2.itertuples():
    a,=row.antecedents
    c,=row.consequents
    if  any(d.isdigit() for d in a[0]):
      age_col.append(a)
      edu_col.append(c)
    else:
      age_col.append(c)
      edu_col.append(a)    

In [36]:
plot_frame=pd.DataFrame(list(zip(age_col,edu_col,vis2.lift)),columns=["age","education_level","lift"])

In [37]:
plot_frame

,age,education_level,lift
0,0-20,zakladne,1.654299
1,21-40,stredoskolske,1.456272
2,21-40,ucnovske,1.329828
3,21-40,vysokoskolske,1.436865
4,41-60,stredoskolske,1.477494


In [38]:
plot_frame=plot_frame[plot_frame.index.isin(list(plot_frame.groupby("education_level")['lift'].idxmax()))] 

In [ ]:
#zakladne:basic,stredoskolske:high school,vysokoskolske:university,ucnovske:apprentice

In [40]:
import plotly.express as px

fig = px.bar(plot_frame, x="education_level", y="lift", color='age', 
             labels=dict(age="Age Group",education_level="Level of Education Completed"),
            color_discrete_sequence=px.colors.qualitative.Dark2)
fig.update_layout(title_text="Correlation Analysis Between Education and Age")
fig.update_yaxes(range=[1,2])
fig.show()

#Visualization 3

In [41]:
vis3=rules
vis3=vis3.drop_duplicates(subset =["support"] )
vis3.support=vis3.support*100

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
age_col=[]
edu_col=[]

for row in vis3.itertuples():
    a,=row.antecedents
    c,=row.consequents
    if  any(d.isdigit() for d in a[0]):
      age_col.append(a)
      edu_col.append(c)
    else:
      age_col.append(c)
      edu_col.append(a) 

In [43]:
plot_frame=pd.DataFrame(list(zip(age_col,edu_col,vis3.support)),columns=["age","education_level","% of User"])

In [44]:
fig = px.bar(plot_frame, x="education_level", y="% of User",
             labels=dict( Age="Age Group"),
             color='age', barmode='group',
             color_discrete_sequence=px.colors.qualitative.Vivid)
fig.update_layout(title_text="Pattern of Education among Different Age Groups")
fig.show()